# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

**1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval (Fill where ever it says #TODO in this part)** <br>

Let me know if you have any problems with this notebook or implementation. <br>
**Telegram**: [@Mohammad_Ardestani4](https://t.me/Mohammad_Ardestani4) <br>
**Email**: mjavad.ardestani00@gmial.com

In [1]:
!pip install elasticsearch

     |████████████████████████████████| 381 kB 111 kB/s eta 0:00:01
     |████████████████████████████████| 59 kB 5.8 MB/s  eta 0:00:01


In [2]:
from elasticsearch import Elasticsearch, helpers
import os, json, time
from datetime import datetime
from tqdm import tqdm 

## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [3]:
import warnings
warnings.filterwarnings('ignore')

## Load Processed Data

In [4]:
import json
dataset_path = 'IR_data_news_12k.json'
with open(dataset_path) as f:
    data = json.load(f)
print(data['0'].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (localhost:9200 is the default), we have to connect to it via the following piece of code.


In [5]:
index_name = 'my_index_name'
es = Elasticsearch("http://localhost:9200")
es.indices.create(index = index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index_name'})

### Check the Cluster Status

In [7]:
dict(es.info())

{'name': 'Rahas-MacBook-Pro.local',
 'cluster_name': 'elasticsearch_raha',
 'cluster_uuid': 'gRNM8n1qR4iLTGoTrgXe0Q',
 'version': {'number': '7.17.4',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '79878662c54c886ae89206c685d9f1051a9d6411',
  'build_date': '2022-05-18T18:04:20.964345128Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

## Indexing Documents 

we can add our documents to created index in two ways:<br>
1- one by one in for loop. <br>
2- use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.

###  for loop by es.index 
**You dont have to run this cell**. 

In [8]:
for i in tqdm(range(len(data))):
    es.index(index = index_name, id=i, document=data[str(i)])
    i = i + 1

100%|█████████████████████████████████████| 12202/12202 [12:09<00:00, 16.74it/s]


###  Bulk API

In [9]:
from elasticsearch.helpers import bulk

def bulk_sync():
    actions = [
        {
            '_index': 'ta3_index',
            '_id':doc_id,
            '_source': doc
        } for doc_id,doc in data.items()
    ]
    bulk(es, actions)


In [10]:
start = time.time()
bulk_sync()
end = time.time()
print("Indexing all documents took about {:.2f} seconds".format(end - start))

Indexing all documents took about 4.52 seconds


### check index

In [11]:
es.count(index = index_name)

ObjectApiResponse({'count': 12202, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

**must**: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
**filter**: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
**should**: The clause (query) should appear in the matching document.
<br><br>
**must_not**: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 

###  <span style="color:red"> TODO </span>
You should read about [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [match phrase query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html) then complete the below cell.

In [16]:
es.get(index=index_name, id=1)['_source']

{'title': 'سجادی :حضور تماشاگران در  لیگ برتر فوتبال تابع نظر فدراسیون  و سازمان لیگ است',
 'content': '\nبه گزارش خبرگزاری فارس، سید حمید سجادی در حاشیه مراسم گرامیداشت روز جوان در جمع خبرنگاران در رابطه با عرضه سهام سرخابی\u200cها در بورس اظهار داشت: منتظر طی روند هستیم و بعدا اطلاع رسانی خواهیم کرد. وی در مورد حضور تماشاگران در مسابقات فوتبال اظهار داشت:\xa0حضور تماشاگران در\xa0 لیگ برتر فوتبال تابع نظر فدراسیون ،سازمان لیگ و ستاد ملی مبارزه با کرونا است. انتهای پیام/\n\n\n',
 'tags': ['سجادی', 'لیگ', 'فدراسیون', 'وزیر ورزش'],
 'date': '3/15/2022 5:30:07 PM',
 'url': 'https://www.farsnews.ir/news/14001224000982/سجادی-حضور-تماشاگران-در-لیگ-برتر-فوتبال-تابع-نظر-فدراسیون-و-سازمان',
 'category': 'sports'}

In [65]:
query= {
        "bool": {
          "should": [
              { 
                  "match": {
                      "content":"آمریکا"
                  }

              }, 
              
              {
                  "match_phrase": {
                      "content":  "تحریم هسته‌ای"
                  }
              },
   
          ],
          "must_not": [
              {
                  "match": {
                      "content":"ایران"
                  }
              }
          ],
        },
    }

res = es.search(index = index_name, query=query, _source= ["url"])
res = dict(res)
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])

171 results in 0.003 s: 
https://www.farsnews.ir/news/14001214000789/نماینده-کلیمیان-در-مجلس-دهم--کارتل‌ها-بر-تصمیمات-هیأت-حاکمه-آمریکا
https://www.farsnews.ir/news/14001217000665/خباثت‌های-آمریکا-در-برجام-روسیه-و-چین-را-هم-به-این-کشور-بدبین-کرد
https://www.farsnews.ir/news/14001213000089/آمریکا-با-بحران‌آفرینی-به-حیات-خود-ادامه-می-دهد
https://www.farsnews.ir/news/14001211000321/آمریکا-دولت‌های-متحد-خود-را-در-زمان-اضطرار-تنها-می‌گذارد
https://www.farsnews.ir/news/14001211000898/سود-مافیای-اسلحه‌سازی-آمریکا-در-ناامن-بودن-جهان-است
https://www.farsnews.ir/news/14001214000825/آمریکا-رژیمی-مافیایی-است-و-مردم-در-تصمیمات-حاکمان-آن-جایگاهی-ندارند
https://www.farsnews.ir/news/14001212000725/آمریکا-برای-فروش-تسلیحات-خود-به-ایجادناامنی-و-بحران‌آفرینی-نیاز-دارد
https://www.farsnews.ir/news/14001204000444/۲۸-آمریکایی-در-جام-یاشاردوغو-جردن-باروز-هم-می‌آید
https://www.farsnews.ir/news/14001213000328/کارتل‌های-اقتصادی-رؤسای-جمهور-آمریکا-را-تعیین-می‌کنند
https://www.farsnews.ir/news/14001211000220/نمای

In [66]:
query= {
        "bool": {
          "should": [
              { 
                  "match": {
                      "content":"غزه"
                  }

              }
   
          ],
          "must_not": [
              {
                  "match": {
                      "content":"ضدتروریست"
                  }
              }
          ],
        },
    }

res = es.search(index = index_name, query=query, _source= ["url"])
res = dict(res)
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])

12 results in 0.007 s: 
https://www.farsnews.ir/news/14000928000274/اقدام-تحسین-برانگیز-سرمربی-الجزایر-در-حمایت-از-فلسطین-جام-قهرمانی-به
https://www.farsnews.ir/news/14000929000204/بیانیه-بسیج-ورزشکاران-در-محکومیت-اقدام-شرم‌آور-سرمربی-تیم-ملی-امید
https://www.farsnews.ir/news/14000829000236/دبیرخانه-انتفاضه-فلسطین-قرار-گرفتن-جنبش-مقاومت-اسلامی-حماس-در-لیست
https://www.farsnews.ir/news/14000726000200/شش-درخت-زیتون-در-دانشگاه-امام-صادق-ع-کاشته-و-به-یاد-شش-اسیر-فلسطینی
https://www.farsnews.ir/news/14000814000380/واکنش-به-انتقاد-اصلاح‌طلبان-از-باز-پس‌گیری-نفت-کشور-به‌جای-پشتیبانی
https://www.farsnews.ir/news/14001212000749/سفر-تیم-فوتبال-معلولان-فلسطین-به-تهران
https://www.farsnews.ir/news/14001018000306/قم-مهد-انقلاب-و-پایگاه-بصیرت-باقی-خواهد-ماند
https://www.farsnews.ir/news/14001209000540/نگاه-دوگانه-برخی-جریان‌ها-به-یمن-و-اوکراین-نشانه-سرسپردگی‌شان-به
https://www.farsnews.ir/news/14001011000249/پایان-ویژه‌برنامه-کاروان-روایت-فتنه--آغاز-روایتگری-مکتب-سردار
https://www.farsnews.ir/news/1

In [77]:
query= {
        "bool": {
          "should": [
              { 
                  "match": {
                      "content":"آمریکا"
                  }

              }, 
              
              {
                  "match_phrase": {
                      "content":  "تیم ملی کشتی"
                  }
              },
   
          ],
          "must_not": [
              {
                  "match": {
                      "content":"ایران"
                  }
              }
          ],
        },
    }

res = es.search(index = index_name, query=query, _source= ["url"])
res = dict(res)
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])

225 results in 0.004 s: 
https://www.farsnews.ir/news/14001109000631/ویزای-آزادکاران-رسید-زمان-پرواز-به-آمریکا-مشخص-شد
https://www.farsnews.ir/news/14001113000109/باروز-آماده-مبارزه-با-ایران-کشتی‌گیر-مطرح-جهان-وقت-آن-است-صندل‌هایم
https://www.farsnews.ir/news/14001110000196/نگاهی-به-نتایج-رقبای-آمریکایی-آزادکاران-کشورمان-در-جام-یاریگین-رقیب
https://www.farsnews.ir/news/14001026000201/منتخبان-کشتی-آزاد-قهرمانی-کشور-به-اردوی-ملی-دعوت-شدند-احتمال-حضور-در
https://www.farsnews.ir/news/14001102000565/حضور-صالحی‌امیری-در-تمرین-کشتی-آزاد-دبیر-بازی‌های-آسیایی-و-جهانی-هر
https://www.farsnews.ir/news/14000925000678/زیارت-آزادکاران-در-حرمین-امام-موسی-کاظم-ع-و-امام-جواد-ع-تصاویر
https://www.farsnews.ir/news/14001209000337/درستکار-تعدادی-از-کشتی‌گیران-انتظارات-را-برآورده-نکردند-کسب-3-طلا
https://www.farsnews.ir/news/14000926000536/آخرین-فرصت-بنا-به-جاماندگان-لیگ-کشتی
https://www.farsnews.ir/news/14001026000646/برگزاری-چرخه-انتخابی-کشتی-فرنگی-با-5-ورزشکار-و-نظارت-بنا-بر-مسابقات
https://www.farsnews.i

### Search query

The _source field contains the original JSON document body that was passed at index time. The _source field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [42]:
res = es.search(index = index_name, query=query, _source= ["url"])
res = dict(res)

### Results

In [ ]:
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])